# Notice: Execute on GPU P100 on kaggle

# Explanation:
generate_baseline_multihop - execute prompts without patching


evaluate_attriburte_exraction_batch_multihop - the patching method


run_experiment- call the patching for all the combinations of layers

generate_CoT_data_v7 - create the prompts data set


step_by_step_cot_baseline - the chain pf thought method



In [21]:
!git clone https://github.com/PAIR-code/interpretability.git

fatal: destination path 'interpretability' already exists and is not an empty directory.


In [22]:
%cd interpretability/patchscopes/code
!pwd


/content/interpretability/patchscopes/code
/content/interpretability/patchscopes/code


In [23]:
import torch
print(torch.cuda.device_count())
# Step 3: Install/upgrade necessary packages
!pip install --upgrade transformers huggingface_hub

1


In [24]:
import numpy as np
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# Install all repo requirments

In [25]:
import pkg_resources
import subprocess
import sys

# List of required packages and their versions
required_packages = {
    'datasets': 'datasets==2.14.7',
}

# Function to check and install the required package
def install_package(package_name, package_version):
    try:
        # Only the package name should be used to check its presence, not the version
        pkg_resources.get_distribution(package_name)
        print(f"{package_name} is already installed")
    except pkg_resources.VersionConflict as e:
        print(f"Version conflict: {e}. Attempting to install the correct version.")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_version])
    except pkg_resources.DistributionNotFound:
        # Package not found, so install it
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_version])

# Loop through the required packages and install them if they are not installed
for package, version in required_packages.items():
    install_package(package, version)


datasets is already installed


In [26]:
from ast import literal_eval
import functools
import json
import os
import random
import shutil

# Scienfitic packages
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
import torch
import datasets
from torch import cuda
import tqdm
torch.set_grad_enabled(False)

# Visuals
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(context="notebook",
        rc={"font.size":16,
            "axes.titlesize":16,
            "axes.labelsize":16,
            "xtick.labelsize": 16.0,
            "ytick.labelsize": 16.0,
            "legend.fontsize": 16.0})
palette_ = sns.color_palette("Set1")
palette = palette_[2:5] + palette_[7:]
sns.set_theme(style='whitegrid')

# Utilities

from general_utils import (
  ModelAndTokenizer,
  make_inputs,
  decode_tokens,
  find_token_range,
  predict_from_input,
)

from patchscopes_utils import *

from tqdm import tqdm
tqdm.pandas()

In [27]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
import torch
import general_utils


# Step 1: Set the Hugging Face token as an environment variable
token = "INSERT_HUGGING_FACE_TOKEB"
os.environ["HUGGINGFACE_TOKEN"] = token

# Step 2: Authenticate
login(token=os.environ["HUGGINGFACE_TOKEN"])



# Step 4: Set model and device
model_name = "meta-llama/Llama-2-7b-chat-hf"
my_device = torch.device("cuda:0")

# Step 5: Determine model precision
if any(x in model_name for x in ["13b", "12b", "7b"]):
    torch_dtype = torch.float16
else:
    torch_dtype = None

my_device = torch.device("cuda:0")
print(my_device)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_auth_token=token)
model = AutoModelForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=False, torch_dtype=torch_dtype, use_auth_token=token)


mt = ModelAndTokenizer (
    model = model,
    tokenizer = tokenizer,
    low_cpu_mem_usage=False,
    torch_dtype=torch_dtype,
    device=my_device,
)
mt.model.to(my_device)
print(mt)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful
cuda:0


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ModelAndTokenizer(model: LlamaForCausalLM [32 layers], tokenizer: LlamaTokenizerFast)


In [28]:
def set_hs_patch_hooks_llama_batch_omer(
    model,
    hs_patch_config,
    module="hs",
    patch_input=False,
    generation_mode=False,
):
  """LLAMA patch hooks - supporting batch."""
  # when using mode.generate() the hidden states in the input are cached after
  # the first inference pass, and in the next steps the input/output are of
  # size 1. In these cases we don't need to patch anymore the previous hidden
  # states from the initial input, because they are cached, but we do need to
  # handle these cases in this call because this hook wraps the generation call.
  #
  # NOTE: To use generation mode, we must patch a position that is not the
  # first one. This is because in this case we don't know during generation if
  # we are handling the initial input or a future step and thus don't know if
  # a patching is needed or not.

  # if generation_mode:
  #     for i in hs_patch_config:
  #         for position_, _ in hs_patch_config[i]:
  #             assert position_ > 0

  if module != "hs":
    raise ValueError("Module %s not yet supported", module)

  def patch_hs(name, position_hs, patch_input, generation_mode):
    def pre_hook(module, inp):
      # inp[0]: (batch, sequence, hidden_state)
      idx_, position_, hs_ = (
          position_hs["batch_idx"],
          position_hs["position_target"],
          position_hs["hidden_rep"],
      )
      input_len = len(inp[0][idx_])
      if generation_mode and input_len == 1:
        return
      inp[0][idx_][position_] = hs_

    def post_hook(module, inp, output):
      idx_, position_, hs_ = (
          position_hs["batch_idx"],
          position_hs["position_target"],
          position_hs["hidden_rep"],
      )
      if "skip_ln" in name:
        # output: (batch, sequence, hidden_state)
        output_len = len(output[idx_])
        if generation_mode and output_len == 1:
          return
        output[idx_][position_] = hs_
      else:
        # output[0]: (batch, sequence, hidden_state)
        output_len = len(output[0][idx_])
        if generation_mode and output_len == 1:
          return
        output[0][idx_][position_] = hs_
    if patch_input:
      return pre_hook
    else:
      return post_hook

  hooks = []

  for item in hs_patch_config:
    i = item["layer_target"]
    skip_final_ln = item["skip_final_ln"]
    if patch_input:
      hooks.append(
          model.model.layers[i].register_forward_pre_hook(
              patch_hs(f"patch_hs_{i}", item, patch_input, generation_mode)
          )
      )
    else:
      # when patching a last-layer representation to the last layer of the same
      # model, the final layer norm is not needed because it was already applied
      # (assuming that the representation for patching was obtained by setting
      # output_hidden_representations to True).
      if skip_final_ln and i == len(model.model.layers) - 1:
        hooks.append(
            model.model.norm.register_forward_hook(
                patch_hs(
                    f"patch_hs_{i}_skip_ln", item, patch_input, generation_mode
                )
            )
        )
      else:
        hooks.append(
            model.model.layers[i].register_forward_hook(
                patch_hs(f"patch_hs_{i}", item, patch_input, generation_mode)
            )
        )

  return hooks

In [29]:
mt.set_hs_patch_hooks = set_hs_patch_hooks_llama_batch_omer
mt.model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

# Auto Patch

# Download dataset

In [30]:
import gc
def clean_memory():
    gc.collect()
    torch.cuda.empty_cache()

clean_memory()

In [ ]:
!pip install gdown

## The repo of the data:
https://github.com/StonyBrookNLP/musique/blob/main/download_data.sh

In [10]:
# Import required libraries
import os
import gdown
import zipfile

# Set variables
zip_name = "musique_v1.0.zip"
file_id = "1tGdADlNjWFaHLeZZGShh2IRcpO6Lv24h"

# Download the file from Google Drive
gdown.download(f"https://drive.google.com/uc?id={file_id}", zip_name, quiet=False)

# Unzip the file
with zipfile.ZipFile(zip_name, 'r') as zip_ref:
    zip_ref.extractall()

# Remove the zip file
os.remove(zip_name)

# Remove __MACOSX directory if it exists
macosx_dir = "__MACOSX"
if os.path.exists(macosx_dir):
    os.system(f'rm -rf {macosx_dir}')


Downloading...
From (original): https://drive.google.com/uc?id=1tGdADlNjWFaHLeZZGShh2IRcpO6Lv24h
From (redirected): https://drive.google.com/uc?id=1tGdADlNjWFaHLeZZGShh2IRcpO6Lv24h&confirm=t&uuid=73a8082a-5250-4f96-9dde-d493fdc7b131
To: /content/interpretability/patchscopes/code/musique_v1.0.zip
100%|██████████| 272M/272M [00:11<00:00, 24.1MB/s]


In [11]:
!ls data

dev_test_singlehop_questions_v1.0.json	musique_ans_v1.0_train.jsonl  musique_full_v1.0_train.jsonl
musique_ans_v1.0_dev.jsonl		musique_full_v1.0_dev.jsonl
musique_ans_v1.0_test.jsonl		musique_full_v1.0_test.jsonl


# Create dataset manually

In [31]:
TRAIN_START_RANGE = 0
TRAIN_END_RANGE = 1024
TEST_START_RANGE = 4 * 1024
TEST_END_RANGE =  TEST_START_RANGE + 1024
TARGET_LAYER = 15

In [32]:
import tqdm
def generate_baseline_multihop(
    mt, df, batch_size=16, max_gen_len=10,
):
    def _generate_baseline_single_batch(batch_df):
        batch_size = len(batch_df)
        cases = [
                 ("baseline_multihop3", "hop3"),
                ]
        results = {}
        for target_col, object_col in cases:

            target_baseline_batch = np.array(batch_df[target_col])
            object_batch = np.array(batch_df[object_col])


            # Step 0: run the the model on target prompt baseline (having the subject token in input rather than patched)
            # The goal of this step is to calculate whether the model works correctly by default, and to calculate surprisal
            inp_target_baseline = make_inputs(mt.tokenizer, target_baseline_batch, mt.device)
            seq_len_target_baseline = len(inp_target_baseline["input_ids"][0])
            output_target_baseline_toks = mt.model.generate(
                inp_target_baseline["input_ids"],
                max_length=seq_len_target_baseline + max_gen_len,
                pad_token_id=mt.model.generation_config.eos_token_id,
            )[:, seq_len_target_baseline:]
            generations_baseline = decode_tokens(mt.tokenizer, output_target_baseline_toks)
            generations_baseline_txt = np.array([" ".join(sample_gen) for sample_gen in generations_baseline])


            is_correct_baseline = np.array([
                (object_batch[i] in generations_baseline_txt[i] or
                 object_batch[i].replace(" ", "") in generations_baseline_txt[i].replace(" ", ""))
                for i in range(batch_size)
            ])
            results.update(
                {
                f"generations_{target_col}": generations_baseline_txt,
                f"is_correct_{target_col}": is_correct_baseline,
                }
            )

        return results

    results = {}
    n_batches = len(df) // batch_size
    if len(df)%batch_size !=0:
        n_batches +=1
    for i in tqdm.tqdm(range(n_batches)):
        cur_df = df.iloc[batch_size * i : batch_size * (i + 1)]
        batch_results = _generate_baseline_single_batch(cur_df)
        for key, value in batch_results.items():
            if key in results:
                results[key] = np.concatenate((results[key], value))
            else:
                results[key] = value

    return results


In [33]:
import pandas as pd
import json
def generate_CoT_data_v7(fname_in="./outputs/preprocessed_data_LRE_CoT/factual_multihop/combined_multihop.pkl",
                         fdir_out="./outputs/preprocessed_data_LRE_CoT/factual_multihop", batch_size=512, max_gen_len=20):
    if not os.path.exists(fdir_out):
        os.makedirs(fdir_out)

    print("Step 1: Read multihop dataset created using LRE data prep...")

    data = []

    # Load the JSONL file
    with open(fname_in, 'r') as f:
        for line in f:
            json_data = json.loads(line)
            data.append(json_data)

    # Create a list to store the formatted data
    formatted_data = []

    # Loop through each entry in the data
    for entry_index in range(len(data)):
        entry = data[entry_index]
        hops = entry.get('question_decomposition', [])
        first_hop_question = hops[0].get('question', None)
        first_hop_answer = hops[0].get('answer', None)
        second_hop_question = hops[1].get('question', None)
        second_hop_answer = hops[1].get('answer', None)

        formatted_entry = {
            'Unnamed: 0': 0,  # This seems to be a placeholder
            'sample_id': entry_index,
            'prompt_source': entry.get('question', None),
            'prompt_target': entry.get('question', None),
            'position_source': -1,  # isn't used
            'position_target': -1,  # isn't used
            'baseline_hop2': first_hop_question, # for visualization
            'baseline_hop3': second_hop_question, # for visualization
            'baseline_multihop3': entry.get('question', None),
            'hop1': first_hop_answer,
            'hop2': second_hop_answer,
            'hop3': entry.get('answer', [None])[0]
        }
        formatted_data.append(formatted_entry)

    df = pd.DataFrame(formatted_data)

    df = df.iloc[TRAIN_START_RANGE:TRAIN_END_RANGE]
    df['hop3'] = df['hop3'].astype(str)
    print(f"len of df is {len(df)}")

    # Step 2: Compute baseline generations
    print("Step 2: Compute baseline generations...")
    eval_results = generate_baseline_multihop(mt, df, batch_size=batch_size, max_gen_len=max_gen_len)
    for key, value in eval_results.items():
        df[key] = list(value)

    return df

300

In [34]:
df = generate_CoT_data_v7(fname_in="./data/musique_full_v1.0_dev.jsonl",
                     fdir_out="./outputs/preprocessed_data_LRE_CoT/factual_multihop",
                     batch_size=32, max_gen_len=20)


Step 1: Read multihop dataset created using LRE data prep...
len of df is 1024
Step 2: Compute baseline generations...


100%|██████████| 32/32 [01:02<00:00,  1.96s/it]


In [35]:
def create_batches(df):
    batches = []
    for layer_source in tqdm.tqdm(range(TARGET_LAYER, TARGET_LAYER + 1), desc="Layer Source Loop"):
        for _, row in df.iterrows():
            target_baseline_batch = np.array([row["prompt_target"]])
            # Step 0: run the the model on target prompt baseline (having the subject token in input rather than patched)
            # The goal of this step is to calculate whether the model works correctly by default, and to calculate surprisal
            inp_target_baseline = make_inputs(mt.tokenizer, target_baseline_batch, mt.device)
            seq_length = len(inp_target_baseline["input_ids"][0])
            for position_source in range(seq_length):
                # for position_target in range(seq_length):
                item = dict(row)
                item.update({
                    "layer_source": layer_source,
                    "layer_target": layer_source - 7,
                    "position_source": position_source,
                    "position_target": position_source
                })
                batches.append(item)
    return batches

In [36]:
def evaluate_attriburte_exraction_batch_multihop(
    mt, df, batch_size=32, max_gen_len=10, transform=None
):
    def _evaluate_attriburte_exraction_single_batch(batch_df):
        batch_size = len(batch_df)
        prompt_source_batch = np.array(batch_df["prompt_source"])
        prompt_target_batch = np.array(batch_df["prompt_target"])
        layer_source_batch = np.array(batch_df["layer_source"])
        layer_target_batch = np.array(batch_df["layer_target"])
        position_source_batch = np.array(batch_df["position_source"])
        position_target_batch = np.array(batch_df["position_target"])

        object_batch = np.array(batch_df["hop3"])

        prompt = make_inputs(mt.tokenizer, prompt_source_batch, mt.device)

        # Adjust position_target to be absolute rather than relative
        inp_target = prompt
        for i in range(batch_size):
            if position_target_batch[i] < 0:
                position_target_batch[i] += len(inp_target["input_ids"][i])

        # Step 1: run the the model on source without patching and get the hidden representations.
        inp_source = prompt
        output_orig = mt.model(**inp_source, output_hidden_states=True)

        # hidden_states size (n_layers, n_sample, seq_len, hidden_dim)
        hidden_rep = [
            output_orig.hidden_states[layer_source_batch[i] + 1][i][
                position_source_batch[i]
            ]
            for i in range(batch_size)
        ]

        if transform is not None:
            for i in range(batch_size):
                print(f"type of hidden_rep[i]: {hidden_rep[i]}")
                hidden_rep[i] = transform(hidden_rep[i])

        # Step 2: do second run on target prompt, while patching the input hidden state.
        hs_patch_config = [
            {
                "batch_idx": i,
                "layer_target": layer_target_batch[i],
                "position_target": position_target_batch[i],
                "hidden_rep": hidden_rep[i],
                "skip_final_ln": (
                    layer_source_batch[i]
                    == layer_target_batch[i]
                    == mt.num_layers - 1
                ),
            }
            for i in range(batch_size)
        ]
        patch_hooks = mt.set_hs_patch_hooks(
            mt.model, hs_patch_config, patch_input=False, generation_mode=True
        )


        output = mt.model(**prompt)

        # NOTE: inputs are left padded,
        # and sequence length is the same across batch
        seq_len = len(inp_target["input_ids"][0])
        output_toks = mt.model.generate(
            inp_target["input_ids"],
            max_length=seq_len + max_gen_len,
            pad_token_id=mt.model.generation_config.eos_token_id,
        )[:, seq_len:]
        generations_patched = decode_tokens(mt.tokenizer, output_toks)
        generations_patched_txt = np.array([
            " ".join(generations_patched[i])
            for i in range(batch_size)
        ])
        is_correct_patched = np.array([
            (object_batch[i] in generations_patched_txt[i]
             or object_batch[i].replace(" ", "") in generations_patched_txt[i].replace(" ", ""))
            for i in range(batch_size)
        ])

        # remove patching hooks
        remove_hooks(patch_hooks)

        cpu_hidden_rep = np.array([hidden_rep[i].detach().cpu().numpy() for i in range(batch_size)])
        if True in is_correct_patched:
            print("is_correct_patched")

        results = {
            "generations_patched": generations_patched,
            "is_correct_patched": is_correct_patched,
            "hidden_rep": cpu_hidden_rep,
        }

        clean_memory()
        del generations_patched_txt, output_toks, output, output_orig, inp_source, inp_target, prompt, hidden_rep

        return results

    results = {}
    n_batches = len(df) // batch_size
    if len(df)%batch_size !=0:
        n_batches +=1
    for i in tqdm.tqdm(range(len(df) // batch_size)):
        clean_memory()
        cur_df = df.iloc[batch_size * i : batch_size * (i + 1)]
        batch_results = _evaluate_attriburte_exraction_single_batch(cur_df)
        for key, value in batch_results.items():
            if key in results:
                results[key] = np.concatenate((results[key], value))
            else:
                results[key] = value

    return results

In [37]:
def run_experiment(df, fname_in, fdir_out, fname_out = "multihop", batch_size=512, n_samples=-1,
                   save_output=True, replace=False):
    print(f"\tNumber of samples: {len(df)}")

    # BATCHED
    batch = create_batches(df)
    experiment_df = pd.DataFrame.from_records(batch)

    if n_samples > 0 and n_samples<len(experiment_df):
        experiment_df = experiment_df.sample(n=n_samples, replace=False, random_state=42).reset_index(drop=True)

    print(f"\tNumber of datapoints for patching experiment: {len(experiment_df)}")

    eval_results = evaluate_attriburte_exraction_batch_multihop(mt, experiment_df, batch_size=batch_size)

    results_df = experiment_df.head(len(eval_results["is_correct_patched"]))
    for key, value in eval_results.items():
        results_df[key] = list(value)

    return results_df

In [38]:
import tqdm
cot_correct_baseline = run_experiment(df,
    f"./outputs/preprocessed_data_LRE_CoT/factual_multihop/combined_multihop_CoT_only_correct_True.pkl",
    "./outputs/results_LRE_CoT/factual_multihop",
    fname_out = f"combined_multihop_CoT_only_correct_True", batch_size=8, n_samples=-1,
    save_output=True, replace=False)

	Number of samples: 1024


Layer Source Loop: 100%|██████████| 1/1 [00:01<00:00,  1.27s/it]


	Number of datapoints for patching experiment: 24919


  0%|          | 3/3114 [00:03<1:01:17,  1.18s/it]

is_correct_patched


...

100%|█████████▉| 3102/3114 [1:05:30<00:15,  1.28s/it]

is_correct_patched


100%|█████████▉| 3103/3114 [1:05:32<00:14,  1.28s/it]

is_correct_patched


100%|██████████| 3114/3114 [1:05:46<00:00,  1.27s/it]
<ipython-input-37-62e39204e041>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df[key] = list(value)
<ipython-input-37-62e39204e041>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df[key] = list(value)
<ipython-input-37-62e39204e041>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

# Train the classifier

In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

classifiers = []

layers_with_classifiers = [15]
for layer_source in layers_with_classifiers:
    # Filter the DataFrame for the current layer_source
    filtered_df = cot_correct_baseline[cot_correct_baseline['layer_source'] == layer_source]

    # Convert the 'hidden_rep' lists into a DataFrame of features
    features = pd.DataFrame(filtered_df['hidden_rep'].tolist())

    # Prepare the target variable
    labels = filtered_df['is_correct_patched']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    from imblearn.combine import SMOTETomek
    sm = SMOTETomek(sampling_strategy=0.5, random_state=42)
    X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)

    # Standardize features by removing the mean and scaling to unit variance
    scaler = StandardScaler()
    X_train_sm = scaler.fit_transform(X_train_sm)
    X_test = scaler.transform(X_test)

    # Create and train the SVM model with RBF (Gaussian) kernel
    classifier = SVC(kernel='rbf', C=10, gamma='scale')
    classifier.fit(X_train_sm, y_train_sm)

    # Make predictions and evaluate the model
    predictions = classifier.predict(X_test)
    print(f"Classification report for layer_source: {layer_source}")
    print(classification_report(y_test, predictions))

    classifiers.append(classifier)

Classification report for layer_source: 15
              precision    recall  f1-score   support

       False       0.84      0.92      0.88      3808
        True       0.64      0.45      0.53      1175

    accuracy                           0.81      4983
   macro avg       0.74      0.69      0.70      4983
weighted avg       0.80      0.81      0.80      4983



 # The patching using classifiers expirement

In [43]:
import pandas as pd
import json
def generate_CoT_data_v7_in_range(min_value ,max_value, fname_in="./outputs/preprocessed_data_LRE_CoT/factual_multihop/combined_multihop.pkl",
                         fdir_out="./outputs/preprocessed_data_LRE_CoT/factual_multihop", batch_size=512, max_gen_len=20):
    if not os.path.exists(fdir_out):
        os.makedirs(fdir_out)

    print("Step 1: Read multihop dataset created using LRE data prep...")

    data = []

    # Load the JSONL file
    with open(fname_in, 'r') as f:
        for line in f:
            json_data = json.loads(line)
            data.append(json_data)

    # Create a list to store the formatted data
    formatted_data = []

    # Loop through each entry in the data
    for entry_index in range(len(data)):
        entry = data[entry_index]
        hops = entry.get('question_decomposition', [])
        first_hop_question = hops[0].get('question', None)
        first_hop_answer = hops[0].get('answer', None)
        second_hop_question = hops[1].get('question', None)
        second_hop_answer = hops[1].get('answer', None)

        formatted_entry = {
            'Unnamed: 0': 0,  # This seems to be a placeholder
            'sample_id': entry_index,
            'prompt_source': entry.get('question', None),
            'prompt_target': entry.get('question', None),
            'position_source': -1,
            'position_target': -1,
            'baseline_hop2': first_hop_question, # for visualization
            'baseline_hop3': second_hop_question, # for visualization
            'baseline_multihop3': entry.get('question', None),
            'hop1': first_hop_answer,
            'hop2': second_hop_answer,
            'hop3': entry.get('answer', [None])[0]
        }
        formatted_data.append(formatted_entry)

    df = pd.DataFrame(formatted_data)

    df = df.iloc[min_value : max_value]
    df['hop3'] = df['hop3'].astype(str)
    print(f"len of df is {len(df)}")

    return df

In [44]:
def evaluate_attriburte_exraction_batch_multihop_with_classifiers(
    classifiers, mt, df, batch_size=32, max_gen_len=10, transform=None
):
    def _evaluate_attriburte_exraction_single_batch(batch_df):
        batch_size = len(batch_df)
        prompt_source_batch = np.array(batch_df["prompt_source"])
        prompt_target_batch = np.array(batch_df["prompt_target"])

        object_batch = np.array(batch_df["hop3"])


        # Adjust position_target to be absolute rather than relative
        inp_target = make_inputs(mt.tokenizer, prompt_target_batch, mt.device)

        # Step 1: run the the model on source without patching and get the hidden representations.
        inp_source = make_inputs(mt.tokenizer, prompt_source_batch, mt.device)
        output_orig = mt.model(**inp_source, output_hidden_states=True)
        layers_with_classifiers = [TARGET_LAYER]
        # hidden_states size (n_layers, n_sample, seq_len, hidden_dim)
        def get_layer_hidden_states(layerNum, index_in_batch):
          return output_orig.hidden_states[layerNum + 1][index_in_batch]

        if transform is not None:
            for i in range(batch_size):
                print(f"type of hidden_rep[i]: {hidden_rep[i]}")
                hidden_rep[i] = transform(hidden_rep[i])

#         # Step 2: do second run on target prompt, while patching the input hidden state.
        # Initialize the hs_patch_config list
        hs_patch_config = []

        # Iterate over the range of batch_size
        for i in range(batch_size):
            for classified_layer in layers_with_classifiers:
                hidden_states = []
                hidden_rep_np = get_layer_hidden_states(classified_layer, i)
                for hidden_state_index in range(len(hidden_rep_np)):
                    hidden_state = hidden_rep_np[hidden_state_index]
                    should_patched_by_the_classifier = classifiers[layers_with_classifiers.index(classified_layer)].predict(hidden_state.cpu().numpy().reshape(1, -1))


                    if should_patched_by_the_classifier:
                      hidden_states.append(hidden_state_index)
                      for layer_diff in [0]:
                        source_layer_index = classified_layer + layer_diff
                        state_to_patch = get_layer_hidden_states(source_layer_index, i)[hidden_state_index]
                        config_dict = {
                              "batch_idx": i,
                              "layer_target": source_layer_index - 7,
                              "position_target": hidden_state_index,
                              "hidden_rep": state_to_patch,
                              "skip_final_ln": (
                                  source_layer_index
                                  == source_layer_index - 7
                                  == mt.num_layers - 1
                              ),
                            }
                        hs_patch_config.append(config_dict)
            print(f"hidden_states : {hidden_states}")

        patch_hooks = mt.set_hs_patch_hooks(
            mt.model, hs_patch_config, patch_input=False, generation_mode=True
        )

        output = mt.model(**inp_target)

        # NOTE: inputs are left padded,
        # and sequence length is the same across batch
        seq_len = len(inp_target["input_ids"][0])
        output_toks = mt.model.generate(
            inp_target["input_ids"],
            max_length=seq_len + max_gen_len,
            pad_token_id=mt.model.generation_config.eos_token_id,
        )[:, seq_len:]
        generations_patched = decode_tokens(mt.tokenizer, output_toks)
        generations_patched_txt = np.array([
            " ".join(generations_patched[i])
            for i in range(batch_size)
        ])
        is_correct_patched = np.array([
            (object_batch[i] in generations_patched_txt[i]
             or object_batch[i].replace(" ", "") in generations_patched_txt[i].replace(" ", ""))
            for i in range(batch_size)
        ])

        # remove patching hooks
        remove_hooks(patch_hooks)

        results = {
            "generations_patched": generations_patched,
            "is_correct_patched": is_correct_patched,
        }
        clean_memory()
        del generations_patched_txt, output_toks, output, output_orig, inp_source, inp_target
        return results

    results = {}
    n_batches = len(df) // batch_size
    if len(df)%batch_size !=0:
        n_batches +=1
    for i in tqdm.tqdm(range(len(df) // batch_size)):
        clean_memory()
        cur_df = df.iloc[batch_size * i : batch_size * (i + 1)]
        batch_results = _evaluate_attriburte_exraction_single_batch(cur_df)

        for key, value in batch_results.items():
            if key in results:
                results[key] = np.concatenate((results[key], value))
            else:
                results[key] = value

    return results

In [45]:
def run_experiment_with_classifiers(classifiers, df, fname_in, fdir_out, fname_out = "multihop", batch_size=512, n_samples=-1,
                   save_output=True, replace=False):
    print(f"\tNumber of samples: {len(df)}")

    experiment_df = df.copy()
    print(f"\tNumber of datapoints for patching experiment: {len(experiment_df)}")
    eval_results = evaluate_attriburte_exraction_batch_multihop_with_classifiers(classifiers, mt, experiment_df, batch_size=batch_size)
    results_df = experiment_df.head(len(eval_results["is_correct_patched"]))

    for key, value in eval_results.items():
        results_df[key] = list(value)

    if save_output:
        if not os.path.exists(fdir_out):
            os.makedirs(fdir_out)
        results_df.to_csv(f"{fdir_out}/{fname_out}.tsv", sep="\t")
        results_df.to_pickle(f"{fdir_out}/{fname_out}.pkl")

    return results_df

In [46]:
test_df = generate_CoT_data_v7_in_range(TRAIN_START_RANGE,
TRAIN_END_RANGE, fname_in="./data/musique_full_v1.0_dev.jsonl",
                     fdir_out="./outputs/preprocessed_data_LRE_CoT/factual_multihop",
                     batch_size=4, max_gen_len=20)

Step 1: Read multihop dataset created using LRE data prep...
len of df is 1024


In [57]:
import tqdm
df_classifier = test_df.copy()
cot_correct_baseline_with_classifiers = run_experiment_with_classifiers(classifiers, test_df, "./outputs/factual/multihop_product_company_ceo.pkl",
               "./outputs/results/factual",
               fname_out = "multihop_product_company_ceo", batch_size=8, n_samples=-1,
               save_output=True, replace=False)

	Number of samples: 1024
	Number of datapoints for patching experiment: 1024


  0%|          | 0/128 [00:00<?, ?it/s]

hidden_states : [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
hidden_states : [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
hidden_states : [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
hidden_states : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
hidden_states : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
hidden_states : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
hidden_states : [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
hidden_states : [0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28]


  1%|          | 1/128 [00:12<26:50, 12.68s/it]

hidden_states : [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
hidden_states : [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
hidden_states : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
hidden_states : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
hidden_states : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
hidden_states : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
hidden_states : [0, 1, 

...

 98%|█████████▊| 126/128 [33:49<00:30, 15.22s/it]

hidden_states : [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
hidden_states : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
hidden_states : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 30]
hidden_states : [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
hidden_states : [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
hidden_states : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
hidden_states : [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
hidden_states : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 

 99%|█████████▉| 127/128 [34:03<00:14, 14.70s/it]

hidden_states : [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
hidden_states : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
hidden_states : [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
hidden_states : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
hidden_states : [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
hidden_states : [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
hidden_states : [0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]
hidden_states : [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]


100%|██████████| 128/128 [34:16<00:00, 16.06s/it]
<ipython-input-45-282d4ffb0362>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df[key] = list(value)
<ipython-input-45-282d4ffb0362>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df[key] = list(value)


In [58]:
cot_correct_baseline_with_classifiers

Unnamed: 0  sample_id  \
0              0          0   
1              0          1   
2              0          2   
3              0          3   
4              0          4   
...          ...        ...   
1019           0       1019   
1020           0       1020   
1021           0       1021   
1022           0       1022   
1023           0       1023   

                                          prompt_source  \
0     Who developed the eponymous character from the...   
1     What county is the city that shares a border w...   
2     In what region of the country containing A Lướ...   
3     Where do Greyhound buses leave from in the cit...   
4     Which county does Lloyd Dane's birthplace belo...   
...                                                 ...   
1019  Who runs the school system in the country on t...   
1020  How many square miles is the city where the Yo...   
1021  What county is the city that shares a border w...   
1022  Where did the author of The Canon of Medicine ...   
1023  When was the region immediately north of the r...   

                                          prompt_target  position_source  \
0     Who developed the eponymous character from the...               -1   
1     What county is the city that shares a border w...               -1   
2     In what region of the country containing A Lướ...               -1   
3     Where do Greyhound buses leave from in the cit...               -1   
4     Which county does Lloyd Dane's birthplace belo...               -1   
...                                                 ...              ...   
1019  Who runs the school system in the country on t...               -1   
1020  How many square miles is the city where the Yo...               -1   
1021  What county is the city that shares a border w...               -1   
1022  Where did the author of The Canon of Medicine ...               -1   
1023  When was the region immediately north of the r...               -1   

      position_target                                      baseline_hop2  \
0                  -1  What series is Mickey's Safari in Letterland f...   
1                  -1          In which state is Zubly Cemetery located?   
2                  -1                                  A Lưới >> country   
3                  -1                          Arna Selznick >> employer   
4                  -1                       Lloyd Dane >> place of birth   
...               ...                                                ...   
1019               -1                         Who hosted the tournament?   
1020               -1                    Who was the edict addressed to?   
1021               -1                   Andrew Deveaux >> place of birth   
1022               -1                   Who wrote The Canon of Medicine?   
1023               -1                                    Urim >> country   

                                          baseline_hop3  \
0                                    Who developed #1 ?   
1            What city became the state capital of #1 ?   
2                           John Phan >> place of birth   
3                           #1 >> headquarters location   
4     #1 >> located in the administrative territoria...   
...                                                 ...   
1019                                   A Don >> country   
1020        Where did the Yongle Emperor greet the #1 ?   
1021         What city became the state capital of #1 ?   
1022  Where did #1 think Venus was in relation to th...   
1023                               Where is #1 located?   

                                     baseline_multihop3            hop1  \
0     Who developed the eponymous character from the...    Mickey Mouse   
1     What county is the city that shares a border w...  South Carolina   
2     In what region of the country containing A Lướ...         Vietnam   
3     Where do Greyhound buses leave from in the cit...         Nelvana   
4     Which co

In [59]:
print("General Patching MultiHop Accuracy (all source layer x target layer): ",
      cot_correct_baseline_with_classifiers["is_correct_patched"].mean())

General Patching MultiHop Accuracy (all source layer x target layer):  0.236328125


# Comparing

Compare with COT:

In [50]:
def step_by_step_cot_baseline(df,
    fname_in=f"./preprocessed_data/factual_multihop/combined_multihop_CoT_only_correct_True.pkl",
    fdir_out="./outputs/results_CoT/factual_multihop",
    fname_out = f"combined_multihop_CoT_only_correct_True_step_by_step",
    batch_size=8,
    max_gen_len=20,
    rewrite=False,
    target_col = "baseline_multihop3",
    object_col = "hop3",
    cot_prefix = "Let's think step by step. "):

    df["cot_prefix"] = cot_prefix

    def _generate_baseline_single_batch(batch_df):
        batch_size = len(batch_df)

        results = {}
        target_baseline_batch = np.array(batch_df[target_col])
        target_baseline_batch = np.core.defchararray.add(cot_prefix, target_baseline_batch.astype(str))
        object_batch = np.array(batch_df[object_col])

        inp_target_baseline = make_inputs(mt.tokenizer, target_baseline_batch, mt.device)
        seq_len_target_baseline = len(inp_target_baseline["input_ids"][0])
        output_target_baseline_toks = mt.model.generate(
            inp_target_baseline["input_ids"],
            max_length=seq_len_target_baseline + max_gen_len,
            pad_token_id=mt.model.generation_config.eos_token_id,
        )[:, seq_len_target_baseline:]
        generations_baseline = decode_tokens(mt.tokenizer, output_target_baseline_toks)
        generations_baseline_txt = np.array([" ".join(sample_gen) for sample_gen in generations_baseline])


        is_correct_baseline = np.array([
            (object_batch[i] in generations_baseline_txt[i] or
             object_batch[i].replace(" ", "") in generations_baseline_txt[i].replace(" ", ""))
            for i in range(batch_size)
        ])
        results.update(
            {
            f"step_by_step_generations_{target_col}": generations_baseline_txt,
            f"step_by_step_is_correct_{target_col}": is_correct_baseline,
            }
        )

        return results

    results = {}
    n_batches = len(df) // batch_size
    if len(df)%batch_size !=0:
        n_batches +=1
    for i in tqdm.tqdm(range(n_batches)):
        cur_df = df.iloc[batch_size * i : batch_size * (i + 1)]
        batch_results = _generate_baseline_single_batch(cur_df)
        for key, value in batch_results.items():
            if key in results:
                results[key] = np.concatenate((results[key], value))
            else:
                results[key] = value

    for key, value in results.items():
        df[key] = list(value)

    return df

In [51]:
cot_correct_baseline_step_by_step_baseline = step_by_step_cot_baseline(test_df,
    fname_in=f"./outputs/preprocessed_data_LRE_CoT/factual_multihop/combined_multihop_CoT_only_correct_True.pkl",
    fdir_out="./outputs/results_LRE_CoT/factual_multihop",
    fname_out = f"combined_multihop_CoT_only_correct_True_step_by_step",
    batch_size=8,
    max_gen_len=20,
    target_col = "baseline_multihop3",
    object_col = "hop3",
    cot_prefix = "Let's think step by step. ",
    rewrite=True)

100%|██████████| 128/128 [02:52<00:00,  1.35s/it]


In [52]:
print("Canonical CoT ('Let's think step by step. ') MultiHop Accuracy: ",
      cot_correct_baseline_step_by_step_baseline.groupby(['sample_id'])["step_by_step_is_correct_baseline_multihop3"].max().reset_index()["step_by_step_is_correct_baseline_multihop3"].mean())

Canonical CoT ('Let's think step by step. ') MultiHop Accuracy:  0.2744140625


In [53]:
import tqdm
def generate_baseline_multihop(
    mt, df, batch_size=256, max_gen_len=10,
):
    def _generate_baseline_single_batch(batch_df):
        batch_size = len(batch_df)
        cases = [
                 ("baseline_multihop3", "hop3"),
                ]
        results = {}
        for target_col, object_col in cases:

            target_baseline_batch = np.array(batch_df[target_col])
            object_batch = np.array(batch_df[object_col])


            # Step 0: run the the model on target prompt baseline (having the subject token in input rather than patched)
            # The goal of this step is to calculate whether the model works correctly by default, and to calculate surprisal
            inp_target_baseline = make_inputs(mt.tokenizer, target_baseline_batch, mt.device)
            seq_len_target_baseline = len(inp_target_baseline["input_ids"][0])
            output_target_baseline_toks = mt.model.generate(
                inp_target_baseline["input_ids"],
                max_length=seq_len_target_baseline + max_gen_len,
                pad_token_id=mt.model.generation_config.eos_token_id,
            )[:, seq_len_target_baseline:]
            generations_baseline = decode_tokens(mt.tokenizer, output_target_baseline_toks)
            generations_baseline_txt = np.array([" ".join(sample_gen) for sample_gen in generations_baseline])


            is_correct_baseline = np.array([
                (object_batch[i] in generations_baseline_txt[i] or
                 object_batch[i].replace(" ", "") in generations_baseline_txt[i].replace(" ", ""))
                for i in range(batch_size)
            ])
            results.update(
                {
                f"generations_{target_col}": generations_baseline_txt,
                f"is_correct_{target_col}": is_correct_baseline,
                }
            )

        return results

    results = {}
    n_batches = len(df) // batch_size
    if len(df)%batch_size !=0:
        n_batches +=1
    for i in tqdm.tqdm(range(n_batches)):
        cur_df = df.iloc[batch_size * i : batch_size * (i + 1)]
        batch_results = _generate_baseline_single_batch(cur_df)
        for key, value in batch_results.items():
            if key in results:
                results[key] = np.concatenate((results[key], value))
            else:
                results[key] = value

    return results


In [54]:
df_base = test_df.copy()
max_gen_len = 10
batch_size = 8
eval_results = generate_baseline_multihop(mt, df_base, batch_size=batch_size, max_gen_len=max_gen_len)
for key, value in eval_results.items():
    df_base[key] = list(value)

100%|██████████| 128/128 [01:31<00:00,  1.40it/s]


In [55]:
print("Base MultiHop Accuracy: ",
      df_base.groupby(['sample_id'])["is_correct_baseline_multihop3"].max().reset_index()["is_correct_baseline_multihop3"].mean())

Base MultiHop Accuracy:  0.1845703125
